## RNN 모델 N-Byte 방식 (함수정보 포함 vs 미포함 => 1:1 비율)

## (1) 데이터로드

In [5]:
# (1) 데이터로드
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings(action='ignore')

# 여러개 쳐도 나오게
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

# 파일읽기
bin8_0 = pd.read_csv("../data/gcc8/o0123합본/o0/o0_gcc8.csv", index_col=0)
core8_0 = pd.read_csv("../data/gcc8/o0123합본/o0/o0_core8.csv", index_col=0)
print(bin8_0.shape, core8_0.shape)

# reset_index (hex processing 하면서 값이 빠졌으니까 + n_gram 에서 index를 다루기 때문에)
bin8_0.reset_index(inplace=True, drop=True)
core8_0.reset_index(inplace=True, drop=True)

print('bin - core concat')
bin8_0 = pd.concat([bin8_0, core8_0], axis=0)
print('shape -', bin8_0.shape)
print('reset_index 완료')
print('input data shape')
bin8_0.head()

(22459287, 2) (7651934, 2)
bin - core concat
shape - (30111221, 2)
reset_index 완료
input data shape


,bin,label
0,47,0
1,108,0
2,105,0
3,98,0
4,47,0


In [6]:
# (2-1) 데이터체크 1 - hex(16진수)가 256 label을 가져야 dummies 변환 가능 
# 16진수 256개 종류가 있어서 pd.get_dummies 사용 가능.
print(len(bin8_0['bin'].unique()))

# (2-2) 데이터 체크 2 - 1, 0 비율 ==> 1이 함수의 갯수를 뜻함
# 정답 데이터 1, 0 비율 확인  ==> 1이 함수의 갯수를 뜻함
print(bin8_0['label'].value_counts())

256
0    30060985
1       50236
Name: label, dtype: int64


## (3) N Byte씩 자르기

In [8]:
idx_bin = bin8_0[bin8_0['label']==1].index  # 407, 474 ...
ls_bin = list(idx_bin)

# 최종 뽑을 행에 대한 index
ls_idx_bin = []

# n byte 자르기 방식
left_idx, right_idx = 0, 32 # 3개씩

# n byte 자르기
for k in range(left_idx, right_idx):
    ls_idx_bin.extend(list(idx_bin + k)) # index 형이라서 가능

#ls_idx = list(set(ls_idx)) 
ls_idx_bin.sort() # 인덱스 정렬

# 1차 index 해당범위 초과한 것들 없애기
ls_idx_bin = list(filter(lambda x: x<len(bin8_0), ls_idx_bin))
print(len(ls_idx_bin))

# 2차 남은 index들 중 right_idx 나눈 나머지 없애기
sub_bin = len(ls_idx_bin)%(right_idx)
print('나머지', sub_bin)

ls_idx_bin = ls_idx_bin[:len(ls_idx_bin)-sub_bin]
print('최종 길이', len(ls_idx_bin))

print('bin8_0', len(ls_idx_bin))

# loc 로 수정필요
bin8_0_Ngram = bin8_0.loc[ls_idx_bin,:].copy()

1607552
나머지 0
최종 길이 1607552
bin8_0 1607552


## (4) false data 만들기

In [9]:
# false data 만들기 - False 데이터 랜덤 생성

# 목표치
goal_bin = len(bin8_0_Ngram)/right_idx
count_bin = 0

print(goal_bin)

# 최종 데이터 Frame
d_bin = pd.DataFrame(columns = bin8_0.columns)

binutils_df = []
# goal 에 도달할 때까지
while True:
    if (count_bin == goal_bin):
            break
    # 진행상황 살펴보기 위함
            
    # 랜덤 N 바이트씩 뽑음
    # random index
    random_idx_bin = np.random.randint(len(bin8_0)-right_idx)

    if count_bin % 1000==0:
        print(count_bin, end=' ')
        print(random_idx_bin)

    df_bin = bin8_0[random_idx_bin : random_idx_bin + right_idx]
    
    # 뽑은 index의 N 바이트 중에 1이 없는 경우만
    if 1 not in df_bin['label'] and count_bin < goal_bin:
        binutils_df.append(df_bin)
        count_bin+=1

print('완료')
print(len(binutils_df))

79730.0
0 5641357
1000 20617765
2000 3135837
3000 296860
4000 6991429
5000 3321161
6000 16696849
7000 11277614
8000 19343386
9000 3540677
10000 5987687
11000 19196370
12000 4417180
13000 30053811
14000 13704746
15000 29541214
16000 2507364
17000 20060781
18000 2566294
19000 20765284
20000 5954604
21000 11556967
22000 5699246
23000 25996872
24000 22175628
25000 91623
26000 19856690
27000 29734071
28000 22428227
29000 28230694
30000 6566694
31000 29705924
32000 25450380
33000 10923060
34000 3185192
35000 22576347
36000 17372758
37000 22984726
38000 5417476
39000 27536697
40000 4207880
41000 21251463
42000 1906844
43000 26063241
44000 15901862
45000 22789383
46000 29879886
47000 10332272
48000 13366881
49000 18588849
50000 11726448
51000 22865145
52000 1587284
53000 23536007
54000 14844321
55000 8737843
56000 27997288
57000 7765528
58000 28311167
59000 8519618
60000 20269516
61000 27450386
62000 12592714
63000 24232252
64000 12950
65000 25134962
66000 20969053
67000 21602444
68000 1729012

In [10]:
# True data와 False Data 같은지 체크
print(len(binutils_df))
print(bin8_0['label'].value_counts()[1])

79730
50236


## (5) False Data + True Data 합치기

In [11]:
f_data = pd.concat(binutils_df)
final = pd.concat([f_data, bin8_0_Ngram])
final.shape

(5102720, 2)

## (6) one hot encoding

In [12]:
# 훈련데이터 (gcc 최적화버전 0, 1, 2, 3 one hot encoding)
bc8_0_onehot_Ngram = pd.get_dummies(final['bin'])
bc8_0_onehot_Ngram = pd.concat([final['label'], bc8_0_onehot_Ngram], axis=1)

print('원핫인코딩완료')
print(bc8_0_onehot_Ngram.shape)

원핫인코딩완료
(5102720, 257)


In [13]:
# 훈련 데이터, 훈련 라벨
x_bc8_0 = bc8_0_onehot_Ngram.iloc[:,1:].to_numpy()
y_bc8_0 = bc8_0_onehot_Ngram['label'].to_numpy()
print(x_bc8_0.shape, x_bc8_0.shape)

x_bc8_0 = x_bc8_0.reshape(-1, right_idx, x_bc8_0.shape[1])
y_bc8_0 = y_bc8_0.reshape(-1, right_idx, 1)

print(x_bc8_0.shape, y_bc8_0.shape)

(5102720, 256) (5102720, 256)
(159460, 32, 256) (159460, 32, 1)


In [14]:
# numpy 행, 열 섞기
p = np.random.permutation(x_bc8_0.shape[0])

x_bc8_0 = x_bc8_0[p]
y_bc8_0 = y_bc8_0[p]

print(x_bc8_0.shape, y_bc8_0.shape)

(159460, 32, 256) (159460, 32, 1)


## (7) 모델

In [15]:
# (10) 양방향 LSTM 모델링 작업
from tensorflow.keras import layers, models
#from tf.keras.models import Model, Sequential
#from tf.keras.layers import SimpleRNN, Input, Dense, LSTM
#from tf.keras.layers import Bidirectional, TimeDistributed

# 학습
from tensorflow.keras.callbacks import EarlyStopping
early_stopping = EarlyStopping(patience = 3) # 조기종료 콜백함수 정의

xInput = layers.Input(batch_shape=(None,right_idx, 256)) 
xBiLstm = layers.Bidirectional(layers.LSTM(48, return_sequences=True, stateful=False), merge_mode = 'concat')(xInput)
xOutput = layers.TimeDistributed(layers.Dense(1, activation ='sigmoid'))(xBiLstm) # 각 스텝에서 cost가 전송되고, 오류가 다음 step으로 전송됨.

## (8) 학습 - 10 KFold

In [16]:
# 교차검증 kfold
from sklearn.model_selection import KFold

# Accuracy, Precision, Recall, F1-Score
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score

# Confusion Matrix, ROC Curve
from sklearn.metrics import confusion_matrix, roc_auc_score

# 최종 평가지표들 평균용
accuracy, recall, precision, f1score, cm = [], [], [], [], []

# 11. 교차검증 kfold - k.split - 10회 / K-Fold 객체 생성
# kf = KFold(n_splits=10, shuffle=False, random_state=None) # KFold non shuffle 버전
kf = KFold(n_splits=10, shuffle=True, random_state=None) # KFold non shuffle 버전

for train, validation in kf.split(x_bc8_0, y_bc8_0):
    model1 = models.Model(xInput, xOutput)
    model1.compile(loss='binary_crossentropy', optimizer='rmsprop', metrics=['accuracy'])
    model1.summary()
    print('======Training stage======')
    model1.fit(x_bc8_0[train],
               y_bc8_0[train],
               epochs = 10,
               batch_size = 128,
               callbacks=[early_stopping])
    #k_accuracy = '%.4f' %(model1.evaluate(data_10000x[validation], data_10000y[validation])[1])

# 12. 교차검증결과 predict - 검증셋들
    # predict 값
    k_pr = model1.predict(x_bc8_0[validation])
    
    # 테스트 predict 결과들 비교 (평가지표 보기위함)
    pred = np.round(np.array(k_pr).flatten().tolist())
    y_test = np.array(y_bc8_0[validation]).flatten().tolist()
    
# 13. 평가지표들 출력
    ## 평가지표들
    k_accuracy = float(accuracy_score(y_test, pred))
    k_recall =  float(recall_score(y_test, pred))
    k_precision = float(precision_score(y_test, pred))
    k_f1_score = float(f1_score(y_test, pred))
    #k_cm = float(confusion_matrix(y_test, pred))
    
    print('accuracy_score', k_accuracy)
    print('recall_score', k_recall)
    print('precision_score', k_precision)
    print('f1_score', k_f1_score)
    #print('\nconfusion_matrix\n', k_cm)

    accuracy.append(k_accuracy)
    recall.append(k_recall)
    precision.append(k_precision)
    f1score.append(k_f1_score)
    #cm.append(k_cm)
#    print('roc_curve 면적', roc_auc_score(y_test, pred))

# 최종 결과지표
print('\nK-fold cross validation Accuracy: {}'.format(accuracy))
print('\nK-fold cross validation Recall: {}'.format(recall))
print('\nK-fold cross validation Precision: {}'.format(precision))
print('\nK-fold cross validation F1-Score: {}'.format(f1score))
#print('\nK-fold cross validation ConfusionMatrix: {}'.format(cm))

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 20s 137us/sample - loss: 0.0128 - accuracy: 0.9970
Epoch 2/10
143514/143514 [==============================] - 9s 59us/sample - loss: 7.0204e-04 - accuracy: 0.9998
Epoch 3/10
143514/143514 [==============================] - 9s 61us/sample - loss: 4.8180e-04 - accuracy: 0.99

accuracy_score 0.9999392480872946
recall_score 0.9982864976010967
precision_score 0.9964084145715751
f1_score 0.9973465719421382
Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 16s 110us/sample - loss: 1.8432e-04 - accuracy: 0.9999
Epoch 2/10
143514/143514 [==============================] - 9s 60us/sample - loss: 1.674

accuracy_score 0.9999470870437728
recall_score 0.9986089375760737
precision_score 0.9967025338424158
f1_score 0.9976548249804568
Model: "model_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 15s 102us/sample - loss: 1.1810e-04 - accuracy: 1.0000
Epoch 2/10
143514/143514 [==============================] - 9s 60us/sample - loss: 9.368

accuracy_score 0.9999686441740876
recall_score 0.9979825151311366
precision_score 0.9993265993265993
f1_score 0.9986541049798117
Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 15s 102us/sample - loss: 6.3626e-05 - accuracy: 1.0000
Epoch 2/10
143514/143514 [==============================] - 9s 61us/sample - loss: 4.889

accuracy_score 0.9999745233914461
recall_score 0.9998297292695386
precision_score 0.9979605710401088
f1_score 0.9988942757506166
Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 16s 112us/sample - loss: 3.2075e-05 - accuracy: 1.0000
Epoch 2/10
143514/143514 [==============================] - 9s 61us/sample - loss: 1.796

...

accuracy_score 0.9999921610435218
recall_score 0.999663242970197
precision_score 0.999663242970197
f1_score 0.999663242970197
Model: "model_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 15s 103us/sample - loss: 8.1217e-06 - accuracy: 1.0000
Epoch 2/10
143514/143514 [==============================] - 9s 61us/sample - loss: 4.1019e-

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 15s 104us/sample - loss: 3.6230e-06 - accuracy: 1.0000
Epoch 2/10
143514/143514 [==============================] - 9s 61us/sample - loss: 5.3373e-06 - accuracy: 1.0000
Epoch 3/10
143514/143514 [=========

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 17s 119us/sample - loss: 3.3213e-06 - accuracy: 1.0000
Epoch 2/10
143514/143514 [==============================] - 9s 61us/sample - loss: 3.3212e-06 - accuracy: 1.0000
Epoch 3/10
143514/143514 [=========

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0
Model: "model_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 32, 256)]         0         
_________________________________________________________________
bidirectional (Bidirectional (None, 32, 96)            117120    
_________________________________________________________________
time_distributed (TimeDistri (None, 32, 1)             97        
Total params: 117,217
Trainable params: 117,217
Non-trainable params: 0
_________________________________________________________________
======Training stage======
Train on 143514 samples
Epoch 1/10
143514/143514 [==============================] - 15s 102us/sample - loss: 3.3210e-06 - accuracy: 1.0000
Epoch 2/10
143514/143514 [==============================] - 9s 61us/sample - loss: 3.3210e-06 - accuracy: 1.0000
Epoch 3/10
143514/143514 [=========

accuracy_score 1.0
recall_score 1.0
precision_score 1.0
f1_score 1.0

K-fold cross validation Accuracy: [0.9999392480872946, 0.9999470870437728, 0.9999686441740876, 0.9999745233914461, 0.9999921610435218, 0.9999921610435218, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation Recall: [0.9982864976010967, 0.9986089375760737, 0.9979825151311366, 0.9998297292695386, 0.9998293515358362, 0.999663242970197, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation Precision: [0.9964084145715751, 0.9967025338424158, 0.9993265993265993, 0.9979605710401088, 0.9994882292732856, 0.999663242970197, 1.0, 1.0, 1.0, 1.0]

K-fold cross validation F1-Score: [0.9973465719421382, 0.9976548249804568, 0.9986541049798117, 0.9988942757506166, 0.9996587613035318, 0.999663242970197, 1.0, 1.0, 1.0, 1.0]


## (9) 평가지표

In [17]:
print('10-Fold Cross_validation. Accuracy :', np.mean(accuracy))
print('10-Fold Cross_validation. Recall :', np.mean(recall))
print('10-Fold Cross_validation. Precision :', np.mean(precision))
print('10-Fold Cross_validation. F1-Score :', np.mean(f1score))

10-Fold Cross_validation. Accuracy : 0.9999813824783643
10-Fold Cross_validation. Recall : 0.9994200274083879
10-Fold Cross_validation. Precision : 0.9989549591024183
10-Fold Cross_validation. F1-Score : 0.9991871781926752


In [18]:
model1.save('gcc8_bin_core_s32_h48_o0.h5')
print('save 완료')

save 완료
